# DATAPREPROCESSING

In [435]:
def PTLump(data,key):
    ptid_roster = data['PTID_Key'].astype(int)
    ptid_roster = ptid_roster.dropna(how='all') 
    ptid_roster = ptid_roster.unique()
    tbl = pd.concat([pd.DataFrame(data['PTID_Key'].index.values),data['PTID_Key'].astype(int)],axis=1)
    dic = tbl.groupby('PTID_Key').groups
    rowIdx = []
    for i in ptid_roster:
        allDates = data[key].iloc[dic[i]]
        print(i,allDates.idxmax())
        rowIdx.append(allDates.idxmax()) 
    reducX = data.iloc[rowIdx,:]
    reducX = reducX.reset_index(drop=True)
    return reducX

In [436]:
# Logistic Regression

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

# Importing the dataset with library pandas
dataset = pd.read_csv('TADPOLE_InputData.csv')
labels_train = pd.read_csv('TADPOLE_TargetData_train.csv')
labels_test = pd.read_csv('TADPOLE_TargetData_test.csv')
target = pd.read_csv('TADPOLE_PredictTargetData_valid.csv')

# Drop meaningless rows and columns. A good practice is to drop rows before columns.
dataset = dataset[~np.isnan(dataset['PTID_Key'])] # Drop patients with no ID, since they cannot be used for learning or prediction. [] slices the rows in dataframe.
dataset = dataset.dropna(axis=1, how='all') # Drop empty columns in dataset 
 

/Users/jonathanwu/anaconda3/envs/ECE5970/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (85,86,91,92,101,102,103,104,105,106,107,108,109,456,820,1398,1643,1645,1651,1652,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [437]:
#drop all columns with more than 90% of its entries as NaN
nan_threshold = 0.9 * dataset.shape[0]
dataset_stats = dataset.isnull().sum()
col_indexes_to_drop = []
for i in range(len(dataset_stats)):
    if dataset_stats[i] > nan_threshold:
        col_indexes_to_drop.append(i)
dataset.drop(dataset.columns[col_indexes_to_drop],axis=1,inplace=True)
    

In [438]:
#Sort all datasets by ascending patient ID.
dataset = dataset.sort_values('PTID_Key')
labels_train = labels_train.sort_values('PTID_Key')
labels_test = labels_test.sort_values('PTID_Key')
target = target.sort_values('PTID_Key')

# Reindex rows
dataset = dataset.reset_index(drop=True)
labels_train = labels_train.reset_index(drop=True)
labels_test = labels_test.reset_index(drop=True)
target = target.reset_index(drop=True)

# These columns are time stamps that do not carry particular meaning, so dropped for now.
badColumns = ['update_stamp_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
    'update_stamp_UCSFFSX_11_02_15_UCSFFSX51_08_01_16',
    'update_stamp_UCBERKELEYAV45_10_17_16']#,
    #'update_stamp_DTIROI_04_30_14']
dataset = dataset.drop(badColumns,axis=1) # Remove this section from objDataset

# Unfortunately, some numerical columns contain non-numerical character such as '>' in  '>1300'.
# The strategy is to convert these columns to floats by extracting only numbers. For example, '>1300' goes to 1300
columnsObjToNum = ['ABETA_UPENNBIOMK9_04_19_17','TAU_UPENNBIOMK9_04_19_17','PTAU_UPENNBIOMK9_04_19_17']#,'COMMENT_UPENNBIOMK9_04_19_17']
for column in columnsObjToNum:
    colIdx = dataset.columns.get_loc(column)
    rowIdx = np.where(dataset[column].apply(type).values == str)[0] # Find all str type elements in each column, which may or may not contain non-numerical characters such as '<' or '>'.
    for row in rowIdx: # iterate through each row of string type element in the column
        dataset.iloc[row,colIdx] = float(re.sub("[^0-9.]","",dataset[column].values[row])) # Find the float/int number in the string, and cast to float type. 
dataset[columnsObjToNum] = dataset[columnsObjToNum].astype(float) # cast each column to float type

# Convert date columns to date format in dataset, since they are currently imported as object columns
for column in dataset: # variable 'column' is a string
    if dataset[column].dtype == 'object' and dataset[column].str.match('[0-9]+/[0-9]+/[0-9]+').sum() > 0: # returns true if this column contains at least one string that matches date format.
        dataset[column] = pd.to_datetime(dataset[column],format="%m/%d/%y",errors='coerce') # convert string to date


# Some numerical columns contain only one/few possible values, which are more likely to be categorical than numerical features.
# As a result, such columns are converted to objective dtype. e.g. some column with only -4 and nan.
columnsNumToCat = []
for column in dataset:
    psbVal = dataset[column].unique()
    if psbVal.dtype == 'float64' and psbVal.size <= 20:
        columnsNumToCat.append(column)
        validRowIdx = dataset[column].notnull()
        dataset[column].loc[validRowIdx] = dataset[column].loc[validRowIdx].astype(str)
#dataset[columnsNumToCat] = dataset[columnsNumToCat].astype(object) # cast to object columns

# Tally the data types of all data columns, and then separate them according to dtype.
dtypeCounts = dataset.dtypes.value_counts(); # Count the number of columns for each data type. Turns out to be only 'float64' and 'object'.
numDataset = dataset.select_dtypes(include=['float'])
objDataset = dataset.select_dtypes(include=['object'])
dateDataset = dataset.select_dtypes(include=['datetime64']) # select dates from objDataset for variable dateDataset

# Count the number of nan's in each column to get an idea of how sparse each column is. It is very likely to drop sparse columns unless they are highly correlated to results.
nonNanCounts_num = numDataset.count() # returns the count of non-NaN entries for each column in numDataset, since not only we want to impute, we want to know how many we impute, especially for columns with very sparse initial data.
#temp = numDataset.count()/8715
#temp.hist(bins=50)

''' The folloing codes were used to detect the problematic columns (columnsTofix and badColumns)mentioned above
for column in objDataset:
    if sum(objDataset[column].apply(type) == float) -  sum(pd.isnull(objDataset[column])) > 0: # returns true if there is at least one entry that is float but not 'NaN'. 'NaN' are excluded since they are float, but not really numerical.
        print(column)
'''

# Imputing missing data in numDataset
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = 'NaN', strategy = 'mean',axis = 0) # impute numerical columns
imp = imp.fit(numDataset)
numX = imp.transform(numDataset) # Extract data from numData as numX in dtype ndarray
#numAttributes = numDataset.columns.values # Names of columns in numX. Executed after imputation since all NaN columns are dropped.

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
objDataset.loc[-1,:] = np.repeat(np.nan,objDataset.shape[1]) # Append one NaN to the end of each column so that NaN must be a class for each column.
objDataset = objDataset.fillna(value=' ') # LabelEncoder does not work with NaN, so NaN is converted to a space ' ', which is always sorted as the first class by LabelEncoder.
le = LabelEncoder()
catClasses = np.array([]) # initiate an empty list of attribute names
catAttributes = np.array([]) # initiate an empty list of attribute names
for i in range(objDataset.shape[1]):
    objDataset.iloc[:,i] = le.fit_transform(objDataset.iloc[:,i]) # encode column i
    classes = le.classes_ # All the labels in column i, including ' ', which was translated from NaN
    classes[0] = 'NaN' # Replace ' ' with the attribute of the column
    catClasses = np.append(catClasses,classes) # Append column attribute followed by all its labels to catAttribute
    catAttributes = np.append(catAttributes, np.repeat(objDataset.columns.values[i],classes.size))
    
tups = [catAttributes,catClasses]
attrTups = list(zip(*tups)) 
enc = OneHotEncoder(categorical_features = 'all') # Based on numerical categories in objDataset, encode it to one in n-class features. e.g. 0 => 0, 0, 0, ...; 1 => 0, 1, 0, ...; 2 => 0, 0, 1, 0, ...
catDataset = pd.DataFrame(enc.fit_transform(objDataset).toarray()) # Dtype: csr_matrix => numpy array => dataframe
catDataset = catDataset.iloc[:-1,:] # Remove the last line full of NaN added earlier
objDataset = objDataset.iloc[:-1,:] # Remove the last line full of NaN added earlier
multiIdx = pd.MultiIndex.from_tuples(attrTups, names = ['Attribute','Class']) # Construct a two-level column names in the format of catDataset[Attribute][Class]
catDataset.columns = multiIdx 

# Convert dates to numerical, as relative days since the first date in the column for the same patient.
# First build a dictionary between PTID_Key (patient ID) and line indices. THis will also be useful for lumping data.
idxPTIDTable = pd.concat([pd.DataFrame(np.array(range(0,numDataset.shape[0]))),numDataset['PTID_Key']],axis=1)
dic = idxPTIDTable.groupby('PTID_Key').groups

# Each entry of dateX is the number of days since the first date of the same patient in the same column.
dateX = np.zeros(dateDataset.shape)
dateX[:] = np.nan
for key, value in dic.items():
    for i in range(dateDataset.shape[1]): # iterate through each date column
        allDates = dateDataset.iloc[value,i] # Get all dates in column i about patient with ID 'key'
        firstDate = allDates.min()
        validDateIdx = np.where(~pd.isnull(allDates))[0] # indices of all non-NaT dates
        # if not pd.isnull(firstDate): # There is at least one valid date in this column, which is the earliest date of this column
        for j in range(len(validDateIdx)):
            dateX[value[j],i] = (allDates[value[j]]-firstDate).days

# Repacking datasets for ease of inspection
dateDataset = pd.DataFrame(dateX, columns = dateDataset.columns.values) 
numDataset = pd.DataFrame(numX, columns = numDataset.columns.values) # numX is after imputation.
Data = pd.concat([numDataset,catDataset], axis=1) #numDataset is imputed, catDataset does not need imputation since NaN is a class, dateDataset is not imputed.
#XAttributes = Data.columns.values






/Users/jonathanwu/anaconda3/envs/ECE5970/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/jonathanwu/anaconda3/envs/ECE5970/lib/python3.5/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [439]:
# Work on labels_train: convert dates to number of days relative to the initial date, and then bind to the rest of data and do imputation
idxPTIDTable_train = pd.concat([pd.DataFrame(labels_train.index.values),labels_train['PTID_Key']],axis=1)
dic_train = idxPTIDTable_train.groupby('PTID_Key').groups
dateY_train_raw = pd.to_datetime(labels_train['Date'],format="%m/%d/%y",errors='coerce')
dateY_train = np.zeros(labels_train['Date'].size)
dateY_train[:] = np.nan
for key, value in dic_train.items():
    allDates = dateY_train_raw[value] # Get all dates in column i about patient with ID 'key'
    firstDate = allDates.min()
    validDateIdx = np.where(~pd.isnull(allDates))[0]
    for j in range(len(validDateIdx)):
        dateY_train[value[j]] = (allDates[value[j]]-firstDate).days
labels_train['Date'] = dateY_train

# On Training labels: Impute missing data based on most frequent value for each individual patient
imp = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
global_means = labels_train.iloc[:,2:].mean() # will be used when the whole column is missing
for key, value in dic_train.items():
        # Get the most frequent value in each column of each patient
        subLbl = labels_train.iloc[value,:]
        modes = subLbl.mode().iloc[0,:]
        diag = subLbl.iloc[:,2]*4+subLbl.iloc[:,3]*2+subLbl.iloc[:,4]
        diag_mode = diag.mode()
        # If the most frequent element of a column is a nan, that means all elemetns are nan in that column, so we will have to impute missing data in that column from input X.
        if diag_mode.size == 0: # If there is no diagnostic result, impute with the most typical situation
            modes['CN_Diag'] = 0
            modes['MCI_Diag'] = 1
            modes['AD_Diag'] = 0 
        else:
            b = bin(int(diag_mode.iloc[0]))[2:].zfill(3)
            modes['CN_Diag'] = b[0]
            modes['MCI_Diag'] = b[1]
            modes['AD_Diag'] = b[2] 
    
        if np.isnan(modes['ADAS13']):
            modes['ADAS13'] = global_means['ADAS13']
        if np.isnan(modes['Ventricles_Norm']):
            modes['Ventricles_Norm'] = global_means['Ventricles_Norm']
        if np.isnan(modes['MMSE']):
            modes['MMSE'] = global_means['MMSE']

        # impute missing data with either the most frequent item in this column for this patient or population mode if no record is found for this patient.    
        y_train_individual = labels_train.iloc[value,:].append(modes)
        imp = imp.fit(y_train_individual.values[:,2:]) 
        temp = imp.transform(y_train_individual.values[:,2:])
        labels_train.iloc[value,2:] = temp[:-1,:]

colNames = labels_train.columns.tolist()
newColNames = [colNames[1],colNames[0]]+ colNames[2:]
labels_train = labels_train[newColNames]
y_train = labels_train.values
yAttributes = labels_train.columns.values

# Work on labels_test: convert dates to number of days relative to the initial date, and then bind to the rest of data and do imputation
idxPTIDTable_test = pd.concat([pd.DataFrame(labels_test.index.values),labels_test['PTID_Key']],axis=1)
dic_test = idxPTIDTable_test.groupby('PTID_Key').groups
dateY_test_raw = pd.to_datetime(labels_test['Date'],format="%Y-%m-%d",errors='coerce')
dateY_test = np.zeros(dateY_test_raw.size)
dateY_test[:] = np.nan
for key, value in dic_test.items():
    allDates = dateY_test_raw[value] # Get all dates in column i about patient with ID 'key'
    firstDate = allDates.min()
    validDateIdx = np.where(~pd.isnull(allDates))[0]
    for j in range(len(validDateIdx)):
        dateY_test[value[j]] = (allDates[value[j]]-firstDate).days
labels_test['Date'] = dateY_test
colNames = labels_test.columns.tolist()
newColNames = [colNames[1],colNames[0]]+ colNames[2:] # switch PTID_Key to the first column
labels_test = labels_test[newColNames]

# Build a row index lookup table for each patient from Data to labels_train/labels_test
ptid_roster = Data['PTID_Key']
ptid_roster = ptid_roster.dropna(how='all') 
ptid_roster = ptid_roster.unique()
ptid_roster.sort()
ptid_roster = ptid_roster.astype(int)

#train data
ptid_train = labels_train['PTID_Key'].unique()
ptid_train.sort()
#test data
ptid_test = labels_test['PTID_Key'].unique()
ptid_test.sort()
#validation data
ptid_validation = target['PTID_Key'].unique()
ptid_validation.sort()


# Separate input data for traning and test sets 
X_train_rowIdx = []
for i in ptid_train:
    X_train_rowIdx.extend(dic[i].tolist())
    
X_test_rowIdx = []
for i in ptid_test:
    X_test_rowIdx.extend(dic[i].tolist())


    

X_train = Data.iloc[X_train_rowIdx,:]
X_train = X_train.reset_index(drop=True)
X_test = Data.iloc[X_test_rowIdx,:]
X_test = X_test.reset_index(drop=True)




In [440]:
#get last patient visit from input data
patient_visit = {}
for index, row in dataset.iterrows():
#for i in range(len(dataset.values)):
    if row[0] not in patient_visit.keys():
        patient_visit[row[0]] = list()
    patient_visit[row[0]].append(row[1])

first_patient_visit = {}    
for key, val in patient_visit.items():
    first_patient_visit[key] = max(patient_visit[key])

In [441]:
og_train = pd.read_csv('TADPOLE_TargetData_train.csv')
og_test = pd.read_csv('TADPOLE_TargetData_test.csv')
target = pd.read_csv('TADPOLE_PredictTargetData_valid.csv')

#Sort all datasets by ascending patient ID.
og_train = og_train.sort_values('PTID_Key')
og_test = og_test.sort_values('PTID_Key')

# Reindex rows
og_train = og_train.reset_index(drop=True)
og_test = og_test.reset_index(drop=True)

og_train['Date'] = pd.to_datetime(og_train['Date'],format="%m/%d/%y",errors='coerce')
#pd.to_datetime(labels_test['Date'],format="%Y-%m-%d",errors='coerce')
og_test['Date'] = pd.to_datetime(og_test['Date'],format="%Y-%m-%d",errors='coerce')
target['Date'] = pd.to_datetime(target['Date'],format="%m/%d/%y",errors='coerce')

In [442]:
#convert dates in train, test and target to absolute years passed from first visit
abs_days_train = np.zeros((og_train.shape[0],))
for index, row in og_train.iterrows():
    current_ptid = row[1]
    abs_day_conv = row[0] - first_patient_visit[current_ptid]
    #abs_days_train[index] = abs_day_conv.days/360
    abs_days_train[index] = float(abs_day_conv.days/360) - 0.3
    abs_days_train[index] = abs(round(abs_days_train[index]))
og_train['Date'] = abs_days_train

abs_days_test = np.zeros((og_test.shape[0],))
for index, row in og_test.iterrows():
    current_ptid = row[1]
    abs_day_conv = row[0] - first_patient_visit[current_ptid]
    #abs_days_test[index] = abs_day_conv.days/360
    abs_days_test[index] = float(abs_day_conv.days/360) - 0.3
    abs_days_test[index] = abs(round(abs_days_test[index]))
og_test['Date'] = abs_days_test

abs_days_target = np.zeros((target.shape[0],))
for index, row in target.iterrows():
    current_ptid = row[1]
    abs_day_conv = row[0] - first_patient_visit[current_ptid]
    #abs_days_target[index] = abs_day_conv.days/360
    abs_days_target[index] = float(abs_day_conv.days/360) - 0.3
    abs_days_target[index] = abs(round(abs_days_target[index]))
target['Date'] = abs_days_target

In [443]:
#make dates relative to input data
labels_train['Date'] = og_train['Date']
labels_test['Date'] = og_test['Date']

In [444]:
#sort years and space them

#load dictionary key -> y_feats
y_train_dict_years = {}
for index, row in labels_train.iterrows():
    current_ptid = row[0]
    if current_ptid not in y_train_dict_years.keys():
        y_train_dict_years[current_ptid] = list()
    y_train_dict_years[current_ptid].append(row)

time_sorted_y_train = {}
for key, val in y_train_dict_years.items():
    #print(val[0][1])
    time_sorted_y_train[key] = list()
    y_time_list = []
    vals_to_add = len(val)
    for i in range(len(val)):
        y_time_list.append(val[i][1])
    y_time_list.sort()
    #infinite while loop to sort bruteforce
    index = 0
    while(index != vals_to_add):
        for i in range(len(val)):
            if val[i][1] == y_time_list[index]:
                time_sorted_y_train[key].append(val[i])
                break
        index+=1   

In [445]:
time_sorted_y_train[6]

[PTID_Key            6.000000
 Date                0.000000
 CN_Diag             1.000000
 MCI_Diag            0.000000
 AD_Diag             0.000000
 ADAS13              1.000000
 Ventricles_Norm     0.025242
 MMSE               30.000000
 Name: 1, dtype: float64, PTID_Key            6.000000
 Date                1.000000
 CN_Diag             1.000000
 MCI_Diag            0.000000
 AD_Diag             0.000000
 ADAS13              1.000000
 Ventricles_Norm     0.025242
 MMSE               30.000000
 Name: 0, dtype: float64, PTID_Key            6.000000
 Date                1.000000
 CN_Diag             1.000000
 MCI_Diag            0.000000
 AD_Diag             0.000000
 ADAS13              1.000000
 Ventricles_Norm     0.025242
 MMSE               30.000000
 Name: 0, dtype: float64, PTID_Key            6.000000
 Date                3.000000
 CN_Diag             1.000000
 MCI_Diag            0.000000
 AD_Diag             0.000000
 ADAS13              1.000000
 Ventricles_Norm     0.02

In [446]:
#convert train df to time series, ptid_train has all unique 
y_train_num_samples = len(ptid_train)
y_train_cats = list(labels_train.columns)
y_train_cats.remove('Date')
y_train_time_shifted_cats = list()
for cats in y_train_cats:
    if cats == 'PTID_Key':
        y_train_time_shifted_cats.append(cats)
    else:
        y_train_time_shifted_cats.append(str(cats) + '_t')
        y_train_time_shifted_cats.append(str(cats) + '_t+1')
        y_train_time_shifted_cats.append(str(cats) + '_t+2')
        y_train_time_shifted_cats.append(str(cats) + '_t+3')
        y_train_time_shifted_cats.append(str(cats) + '_t+4')
        y_train_time_shifted_cats.append(str(cats) + '_t+5')
y_train_num_feats = len(y_train_time_shifted_cats)

In [447]:
#make dictionary to map ptid -> all rows
y_train_ptid = {}
for index, row in labels_train.iterrows():
    if row[0] not in y_train_ptid.keys():
        y_train_ptid[row[0]] = list()
    y_train_ptid[row[0]].append(row)

In [448]:
#make dictionary that maps ptid -> diagnoses at all timesteps
y_train_timestep = {}
for key, val in y_train_ptid.items():
    y_train_timestep[key] = {}
    #y_train_timestep[key]['Last_TS'] = 0
    y_train_timestep[key]['timestep'] = list()
    y_train_timestep[key]['TS'] = [[],[],[],[],[],[]]
    for entry in val:
        #get timestep
        if int(entry[1]) not in y_train_timestep[key]['timestep']:
            y_train_timestep[key]['timestep'].append(entry[1])
        #if int(entry[1]) > y_train_timestep[key]['Last_TS']:
            #y_train_timestep[key]['Last_TS'] = entry[1]
        y_train_timestep[key]['TS'][int(entry[1])].append(entry)   

In [449]:
#sort entries by timesteps
y_timestep_dict = {}
y_timestep_dict[0] = list()
y_timestep_dict[1] = list()
y_timestep_dict[2] = list()
y_timestep_dict[3] = list()
y_timestep_dict[4] = list()
y_timestep_dict[5] = list()

for index, row in labels_train.iterrows():
    key = int(row[1])
    y_timestep_dict[key].append(row)

In [450]:
condensed_imputed_y_train = {}
for key, val in y_train_timestep.items():
    condensed_imputed_y_train[key] = np.zeros((6,8))
    last_ts = val['timestep']
    all_ts = val['TS']
    #print(len(all_ts))
    #iterate through list of lists
    for i in range(len(all_ts)):
        total_entry = len(all_ts[i])
        
        #if there is no entry for the year
        if total_entry == 0:
            condensed_entry = np.zeros((8,))
            #population mean for target variables
            #last diagnosis patient got/population mode for diagnosis at that timestep
            #years_to_impute = list( set([0,1,2,3,4,5]) - set(y_train_timestep[i]['timestep']) )
            
            #use global means at the timestep to get global averages to impute missing timesteps for patients
            timestep_entries = y_timestep_dict[i]
            running_adas = 0
            running_vn = 0
            running_mmse = 0
            #future_ts_diagnoses = [0,0,0]
            
            for entry in timestep_entries:
                #future_ts_diagnoses[0] += entry[2]
                #future_ts_diagnoses[1] += entry[3]
                #future_ts_diagnoses[2] += entry[4]
                running_adas += entry[5]
                running_vn += entry[6]
                running_mmse += entry[7]
            running_adas /= len(timestep_entries)
            running_vn /= len(timestep_entries)
            running_mmse /= len(timestep_entries)
            condensed_entry[5] = running_adas
            condensed_entry[6] = running_vn
            condensed_entry[7] = running_mmse
            
            #get last year logged
            patient_last_year = max(y_train_timestep[key]['timestep'])
            #if there are missing intermediate values
            if i < patient_last_year:
                ptid_last_visit_entry = all_ts[int(patient_last_year)]
                condensed_entry[2:5] = ptid_last_visit_entry[0][2:5]
                condensed_entry[0] = key
                condensed_entry[1] = i
                #get global means averaged value for patient's vn, 
            #tiestep is from a year after
            else:
                condensed_entry[0] = key
                condensed_entry[1] = i
                ptid_last_visit_entry = all_ts[int(patient_last_year)]
                condensed_entry[2:5] = ptid_last_visit_entry[0][2:5]
                
#                 imputed_diagnoses = list.index(future_ts_diagnoses, max(future_ts_diagnoses))
#                 #CN
#                 if imputed_diagnoses == 0:
#                     condensed_entry[2] = 1
#                     condensed_entry[3] = 0
#                     condensed_entry[4] = 0
#                 #MCI
#                 elif imputed_diagnoses == 1:
#                     condensed_entry[2] = 0
#                     condensed_entry[3] = 1
#                     condensed_entry[4] = 0

#                 #AD
#                 elif imputed_diagnoses == 2:
#                     condensed_entry[2] = 0
#                     condensed_entry[3] = 0
#                     condensed_entry[4] = 1
        #print(total_entry)
        #if theres more than one entry for the year, average over the 
        elif total_entry > 0:
            condensed_entry = np.zeros((8,))
            condensed_entry[0] = key
            condensed_entry[1] = i
            condensed_entry[2:5] = all_ts[i][0][2:5]
            running_adas = 0
            running_vn = 0
            running_mmse = 0

            for entry in all_ts[i]:
                running_adas += entry[5]
                running_vn += entry[6]
                running_mmse += entry[7]

            running_adas /= total_entry
            running_vn /= total_entry
            running_mmse /= total_entry
            condensed_entry[5] = running_adas
            condensed_entry[6] = running_vn
            condensed_entry[7] = running_mmse
        #if theres no entry for the year, use population mean for target variables & last diagnosis patient
        # for values in between
        elif total_entry == 1:
            condensed_entry = np.zeros((8,))
            condensed_entry = all_ts[i][0]
                    
        condensed_imputed_y_train[key][i] = condensed_entry
                    
            

In [451]:
len(condensed_imputed_y_train[6])

6

In [452]:
y_train_time_series = np.zeros((y_train_num_samples, y_train_num_feats))
y_train_ts_ind = 0
for key, val in condensed_imputed_y_train.items():
    set_key = 0
    #print(key)
    #iterate through entries per key
    y_train_entry_array = np.zeros((y_train_num_feats,))
    for entry in val:
        current_timestep = entry[1]
        entry_cn = entry[2]
        entry_mci = entry[3]
        entry_ad = entry[4]
        entry_adas = entry[5]
        entry_vn = entry[6]
        entry_mmse = entry[7]
        #set patient id
        if set_key == 0:
            y_train_entry_array[0] = entry[0]
            set_key = 1
        if current_timestep==0:
            ts_offset = 1
            y_train_entry_array[1] = entry_cn
            y_train_entry_array[7] = entry_mci
            y_train_entry_array[13] = entry_ad
            y_train_entry_array[19] = entry_adas
            y_train_entry_array[25] = entry_vn
            y_train_entry_array[31] = entry_mmse
        elif current_timestep==1:
            y_train_entry_array[2] = entry_cn
            y_train_entry_array[8] = entry_mci
            y_train_entry_array[14] = entry_ad
            y_train_entry_array[20] = entry_adas
            y_train_entry_array[26] = entry_vn
            y_train_entry_array[32] = entry_mmse            

        elif current_timestep==2:
            y_train_entry_array[3] = entry_cn
            y_train_entry_array[9] = entry_mci
            y_train_entry_array[15] = entry_ad
            y_train_entry_array[21] = entry_adas
            y_train_entry_array[27] = entry_vn
            y_train_entry_array[33] = entry_mmse
        elif current_timestep==3:
            y_train_entry_array[4] = entry_cn
            y_train_entry_array[10] = entry_mci
            y_train_entry_array[16] = entry_ad
            y_train_entry_array[22] = entry_adas
            y_train_entry_array[28] = entry_vn
            y_train_entry_array[34] = entry_mmse
        elif current_timestep==4:
            y_train_entry_array[5] = entry_cn
            y_train_entry_array[11] = entry_mci
            y_train_entry_array[17] = entry_ad
            y_train_entry_array[23] = entry_adas
            y_train_entry_array[29] = entry_vn
            y_train_entry_array[35] = entry_mmse
        elif current_timestep==5:
            y_train_entry_array[6] = entry_cn
            y_train_entry_array[12] = entry_mci
            y_train_entry_array[18] = entry_ad
            y_train_entry_array[24] = entry_adas
            y_train_entry_array[30] = entry_vn
            y_train_entry_array[36] = entry_mmse 
    y_train_time_series[y_train_ts_ind] = y_train_entry_array
    y_train_ts_ind +=1

In [453]:
y_train_ts_df = pd.DataFrame(data=y_train_time_series, columns=y_train_time_shifted_cats)

#Sort all datasets by ascending patient ID.
y_train_ts_df = y_train_ts_df.sort_values('PTID_Key')

# Reindex rows
y_train_ts_df = y_train_ts_df.reset_index(drop=True)

In [454]:
y_train_ts_df

,PTID_Key,CN_Diag_t,CN_Diag_t+1,CN_Diag_t+2,CN_Diag_t+3,CN_Diag_t+4,CN_Diag_t+5,MCI_Diag_t,MCI_Diag_t+1,MCI_Diag_t+2,...,Ventricles_Norm_t+2,Ventricles_Norm_t+3,Ventricles_Norm_t+4,Ventricles_Norm_t+5,MMSE_t,MMSE_t+1,MMSE_t+2,MMSE_t+3,MMSE_t+4,MMSE_t+5
0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.024965,0.025242,0.022762,0.015264,30.000000,30.000000,26.341325,29.000000,26.385233,26.0
1,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.024965,0.024257,0.022762,0.015264,30.000000,30.000000,26.341325,26.396504,26.385233,26.0
2,18.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.020526,0.024257,0.022762,0.015264,30.000000,29.500000,30.000000,26.396504,26.385233,26.0
3,21.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.006666,0.006617,0.006617,0.015264,27.000000,28.000000,28.000000,29.000000,27.000000,26.0
4,22.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.024965,0.033158,0.033158,0.015264,27.500000,26.500000,26.341325,28.000000,26.000000,26.0
5,25.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.024965,0.024257,0.022762,0.015264,28.000000,28.000000,26.341325,26.396504,26.385233,26.0
6,26.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.024965,0.013821,0.022762,0.015264,30.000000,30.000000,26.341325,30.000000,26.385233,26.0
7,32.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.024965,0.026180,0.022762,0.015264,30.000000,28.666667,26.341325,28.000000,26.385233,26.0
8,34.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.024965,0.022541,0.022762,0.015264,29.000000,28.000000,26.341325,30.000000,26.385233,26.0
9,36.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.024965,0.024257,0.022762,0.015264,28.500000,26.065176,26.341325,26.396504,26.385233,26.0


# CREATE TIME SERIES INPUT

In [455]:
#mod_numDataset = numDataset.insert(1, 'EXAMDATE', dateDataset.values[:,0])
time_series_data = Data.insert(1, 'Elapsed Days', dateX[:,0] )
#time_series_data = Data.insert(0, 'Elapsed Days', dateX[:,0] )

In [456]:
#create PTID Dictionary, maps PTID -> all visits
PTID_Dict = {}
for i in range(len(Data)):
    if Data.values[i][0] not in PTID_Dict.keys():
        PTID_Dict[Data.values[i][0]] = list()
        PTID_Dict[Data.values[i][0]].append(Data.values[i])
    else:
        PTID_Dict[Data.values[i][0]].append(Data.values[i])

In [457]:
#sort PTID entries by time of visit, val is a list of patient visits
time_sorted_PTID_dict = {}
for key, val in PTID_Dict.items():
    time_sorted_PTID_dict[key] = list()
    time_list = []
    vals_to_add = len(val)
    for i in range(len(val)):
        time_list.append(val[i][1])
    time_list.sort()
    #infinite while loop to sort bruteforce
    index = 0
    while(index != vals_to_add):
        for i in range(len(val)):
            if val[i][1] == time_list[index]:
                time_sorted_PTID_dict[key].append(val[i])
                break
        index+=1
    #print(key)
    #for i in range(len(time_sorted_PTID_dict[key])):
    #    print(time_sorted_PTID_dict[key][i])

In [143]:
#convert time elapsed from days -> years
year_ptid_dict={}
for key,val in time_sorted_PTID_dict.items():
    year_ptid_dict[key] = list()
    for i in range(len(val)):  
        year_ptid_dict[key].append(val[i])
        year_conv = val[i][1]/360
        year_ptid_dict[key][i][1] = year_conv
    #print(key)
    for i in range(len(val)):
        #print(year_ptid_dict[key][i])

1.0
[  1.    0.   81.6 ...,   0.    1.    0. ]
[  1.           0.32777778  81.6        ...,   0.           1.           0.        ]
[  1.           0.75555556  81.6        ...,   0.           1.           0.        ]
2.0
[  2.    0.   76.6 ...,   0.    1.    0. ]
[  2.           0.49722222  76.6        ...,   0.           1.           0.        ]
[  2.      1.025  76.6   ...,   0.      1.      0.   ]
[  2.           1.48611111  76.6        ...,   0.           1.           0.        ]
3.0
[  3.    0.   62.8 ...,   1.    0.    1. ]
[  3.           0.54722222  62.8        ...,   0.           1.           0.        ]
[  3.           1.03055556  62.8        ...,   1.           0.           1.        ]
[  3.           1.52222222  62.8        ...,   0.           1.           0.        ]
[  3.           2.11388889  62.8        ...,   0.           1.           0.        ]
[  3.           2.55277778  62.8        ...,   0.           1.           0.        ]
[  3.           3.12777778  62.8       

[ 23.           0.53611111  68.7        ...,   0.           1.           0.        ]
[ 23.           1.02777778  68.7        ...,   0.           1.           0.        ]
[ 23.           1.58888889  68.7        ...,   0.           1.           0.        ]
24.0
[ 24.    0.   81.4 ...,   1.    0.    1. ]
[ 24.           0.18055556  81.4        ...,   0.           1.           0.        ]
25.0
[ 25.    0.   70.5 ...,   1.    0.    1. ]
26.0
[ 26.    0.   69.8 ...,   1.    0.    1. ]
[ 26.     0.35  69.8  ...,   0.     1.     0.  ]
[ 26.           0.51111111  69.8        ...,   0.           1.           0.        ]
[ 26.           1.03055556  69.8        ...,   0.           1.           0.        ]
27.0
[ 27.    0.   79.4 ...,   1.    0.    1. ]
[ 27.           0.53888889  79.4        ...,   0.           1.           0.        ]
[ 27.           1.04444444  79.4        ...,   1.           0.           1.        ]
[ 27.     1.55  79.4  ...,   0.     1.     0.  ]
[ 27.           2.03333333  79

[ 73.           2.04444444  74.4        ...,   0.           1.           0.        ]
[ 73.           2.56111111  74.4        ...,   0.           1.           0.        ]
[ 73.           3.51666667  74.4        ...,   0.           1.           0.        ]
[ 73.           4.00277778  74.4        ...,   0.           1.           0.        ]
[ 73.           4.58888889  74.4        ...,   0.           1.           0.        ]
[ 73.           6.02777778  74.4        ...,   0.           1.           0.        ]
[ 73.           6.55555556  74.4        ...,   0.           1.           0.        ]
74.0
[ 74.    0.   78.8 ...,   1.    0.    1. ]
75.0
[ 75.    0.   74.2 ...,   0.    1.    0. ]
[ 75.      0.525  74.2   ...,   0.      1.      0.   ]
[ 75.           1.03888889  74.2        ...,   0.           1.           0.        ]
[ 75.           1.53611111  74.2        ...,   0.           1.           0.        ]
[ 75.           2.08888889  74.2        ...,   0.           1.           0.        ]

[ 128.            0.48611111   69.6        ...,    0.            1.            0.        ]
[ 128.            1.01944444   69.6        ...,    0.            1.            0.        ]
[ 128.            1.54444444   69.6        ...,    0.            1.            0.        ]
[ 128.       2.025   69.6   ...,    0.       1.       0.   ]
[ 128.            2.53611111   69.6        ...,    0.            1.            0.        ]
[ 128.            3.01944444   69.6        ...,    0.            1.            0.        ]
[ 128.            3.69444444   69.6        ...,    0.            1.            0.        ]
[ 128.            4.03333333   69.6        ...,    0.            1.            0.        ]
[ 128.            5.03888889   69.6        ...,    0.            1.            0.        ]
129.0
[ 129.     0.    66.2 ...,    1.     0.     1. ]
[ 129.            0.29166667   66.2        ...,    0.            1.            0.        ]
[ 129.            0.85833333   66.2        ...,    0.            

[ 216.     0.    85.8 ...,    0.     1.     0. ]
[ 216.            0.48888889   85.8        ...,    0.            1.            0.        ]
[ 216.            1.00277778   85.8        ...,    0.            1.            0.        ]
[ 216.            1.50277778   85.8        ...,    0.            1.            0.        ]
[ 216.            1.99444444   85.8        ...,    0.            1.            0.        ]
217.0
[ 217.    0.   78. ...,    1.    0.    1.]
[ 217.            0.23055556   78.         ...,    0.            1.            0.        ]
[ 217.            0.51111111   78.         ...,    0.            1.            0.        ]
[ 217.            1.03611111   78.         ...,    0.            1.            0.        ]
218.0
[ 218.     0.    78.9 ...,    0.     1.     0. ]
[ 218.            0.47777778   78.9        ...,    0.            1.            0.        ]
219.0
[ 219.     0.    61.1 ...,    0.     1.     0. ]
[ 219.            0.54166667   61.1        ...,    0.           

...

1284.0
[  1.28400000e+03   0.00000000e+00   8.25000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.28400000e+03   2.13888889e-01   8.25000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.28400000e+03   4.77777778e-01   8.25000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.28400000e+03   9.91666667e-01   8.25000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.28400000e+03   1.53055556e+00   8.25000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.28400000e+03   1.89722222e+00   8.25000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1285.0
[  1.28500000e+03   0.00000000e+00   6.84000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.28500000e+03   2.33333333e-01   6.84000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.28500000e+03   5.08333333e-01   6.84000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+

   1.00000000e+00   0.00000000e+00]
[  1.39500000e+03   4.61388889e+00   7.99000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39500000e+03   5.54444444e+00   7.99000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1396.0
[  1.39600000e+03   0.00000000e+00   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   5.02777778e-01   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   1.00833333e+00   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   1.52500000e+00   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   2.03611111e+00   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   2.55555556e+00   8.22000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.39600000e+03   3.08611111e+00   8.22000000e+01 ...,   0.00000000e+00
   

[  1.47100000e+03   0.00000000e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   5.08333333e-01   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   1.01111111e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   1.51111111e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   2.02222222e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   2.54166667e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.47100000e+03   3.04722222e+00   7.17000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1472.0
[  1.47200000e+03   0.00000000e+00   7.90000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.47200000e+03   1.38888889e-01   7.90000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  

1533.0
[  1.53300000e+03   0.00000000e+00   6.86000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.53300000e+03   1.86111111e-01   6.86000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.53300000e+03   5.05555556e-01   6.86000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.53300000e+03   1.03055556e+00   6.86000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1534.0
[  1.53400000e+03   0.00000000e+00   7.32000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.53400000e+03   5.19444444e-01   7.32000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.53400000e+03   1.00555556e+00   7.32000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.53400000e+03   1.52222222e+00   7.32000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.53400000e+03   2.00000000e+00   7.32000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+

[  1.62400000e+03   0.00000000e+00   8.02000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.62400000e+03   3.08333333e-01   8.02000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.62400000e+03   5.11111111e-01   8.02000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.62400000e+03   1.01666667e+00   8.02000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1625.0
[  1.62500000e+03   0.00000000e+00   7.10000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.62500000e+03   2.72222222e-01   7.10000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.62500000e+03   5.44444444e-01   7.10000000e+01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
1626.0
[  1.62600000e+03   0.00000000e+00   6.52000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
1627.0
[  1.62700000e+03   0.00000000e+00   8.07000000e+01 ...,   1.00000000e+00
   0.00000000e+00   1.000

In [458]:
#get visits that are spaced a year apart
import math
condensed_time_series_dict = {}
for key, val in year_ptid_dict.items():
    condensed_time_series_dict[key] = list()
    last_added = 0
    condensed_time_series_dict[key].append(val[0])
    if len(val)>1:
        for i in range(len(val)-1):
            tmp = val[i+1][1]-.3 #var to check if year is at least .85
            if (round(tmp)) >= (last_added + 1):
                condensed_time_series_dict[key].append(val[i+1])
                last_added = round(tmp)       

In [459]:
#get last two visits of each patient 
last_two_visits_time_series_dict = {}
for key, val in condensed_time_series_dict.items():
    last_two_visits_time_series_dict[key]=list()
    for entry in val[-2:]: 
        last_two_visits_time_series_dict[key].append(np.delete(entry, 1))
    if len(last_two_visits_time_series_dict[key])==1:
        last_two_visits_time_series_dict[key].append(np.delete(val,1))
        
#pack dictionary into dataframe for inspection
last_two_visits_df = pd.DataFrame(data=last_two_visits_time_series_dict)
last_two_visits_df = last_two_visits_df.transpose()
last_two_visits_df.columns = ['t-1', 't']

In [460]:
for key, val in last_two_visits_time_series_dict.items():
    print('PTID:', key)
    for values in last_two_visits_time_series_dict[key]:
        print(values)

PTID: 1.0
[  1.       81.6       0.97265 ...,   0.        1.        0.     ]
[  1.       81.6       0.97265 ...,   0.        1.        0.     ]
PTID: 2.0
[  2.       76.6       1.02232 ...,   0.        1.        0.     ]
[  2.       76.6       1.00526 ...,   0.        1.        0.     ]
PTID: 3.0
[  3.       62.8       1.49546 ...,   0.        1.        0.     ]
[  3.       62.8       1.40991 ...,   0.        1.        0.     ]
PTID: 4.0
[  4.       61.1       1.40864 ...,   1.        0.        1.     ]
[  4.       61.1       1.40864 ...,   1.        0.        1.     ]
PTID: 5.0
[  5.         70.8         1.2045555 ...,   0.          1.          0.       ]
[  5.         70.8         1.2045555 ...,   0.          1.          0.       ]
PTID: 6.0
[  6.         70.4         1.2045555 ...,   0.          1.          0.       ]
[  6.       70.4       1.18324 ...,   1.        0.        1.     ]
PTID: 7.0
[  7.         69.9         1.2045555 ...,   0.          1.          0.       ]
[  7.      

PTID: 296.0
[ 296.        82.3        1.23516 ...,    0.         1.         0.     ]
[ 296.          82.3          1.2045555 ...,    0.           1.           0.       ]
PTID: 297.0
[ 297.        82.8        1.22405 ...,    1.         0.         1.     ]
[ 297.        82.8        1.22969 ...,    0.         1.         0.     ]
PTID: 298.0
[ 298.          86.5          1.2045555 ...,    1.           0.           1.       ]
[ 298.          86.5          1.2045555 ...,    1.           0.           1.       ]
PTID: 299.0
[ 299.        87.3        1.01722 ...,    1.         0.         1.     ]
[ 299.        87.3        1.01722 ...,    1.         0.         1.     ]
PTID: 300.0
[ 300.          77.           1.2045555 ...,    0.           1.           0.       ]
[ 300.          77.           1.2045555 ...,    0.           1.           0.       ]
PTID: 301.0
[ 301.          70.3          1.2045555 ...,    0.           1.           0.       ]
[ 301.          70.3          1.2045555 ...,    0.   

PTID: 569.0
[ 569.        69.3        1.50341 ...,    1.         0.         1.     ]
[ 569.        69.3        1.50341 ...,    1.         0.         1.     ]
PTID: 570.0
[ 570.        69.2        1.42919 ...,    1.         0.         1.     ]
[ 570.        69.2        1.42919 ...,    1.         0.         1.     ]
PTID: 571.0
[ 571.        81.5        1.20844 ...,    1.         0.         1.     ]
[ 571.          81.5          1.2045555 ...,    0.           1.           0.       ]
PTID: 572.0
[ 572.        72.6        1.39671 ...,    1.         0.         1.     ]
[ 572.          72.6          1.2045555 ...,    0.           1.           0.       ]
PTID: 573.0
[ 573.        65.7        1.11292 ...,    1.         0.         1.     ]
[ 573.        65.7        1.11292 ...,    1.         0.         1.     ]
PTID: 574.0
[ 574.        78.         1.03993 ...,    1.         0.         1.     ]
[ 574.        78.         1.03993 ...,    1.         0.         1.     ]
PTID: 575.0
[ 575.          

[ 846.        83.2        1.18501 ...,    1.         0.         1.     ]
[ 846.        83.2        1.18501 ...,    1.         0.         1.     ]
PTID: 847.0
[ 847.          81.4          1.2045555 ...,    0.           1.           0.       ]
[ 847.          81.4          1.2045555 ...,    0.           1.           0.       ]
PTID: 848.0
[ 848.        77.5        1.34012 ...,    0.         1.         0.     ]
[ 848.        77.5        1.29006 ...,    0.         1.         0.     ]
PTID: 849.0
[ 849.         73.5         0.952722 ...,    1.          0.          1.      ]
[ 849.          73.5          1.2045555 ...,    0.           1.           0.       ]
PTID: 850.0
[ 850.          75.2          1.2045555 ...,    0.           1.           0.       ]
[ 850.        75.2        1.18786 ...,    1.         0.         1.     ]
PTID: 851.0
[ 851.        73.4        1.27602 ...,    0.         1.         0.     ]
[ 851.          73.4          1.2045555 ...,    0.           1.           0.       

[  1.09100000e+03   6.26000000e+01   1.20455550e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.09100000e+03   6.26000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1092.0
[  1.09200000e+03   7.91000000e+01   1.02122000e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.09200000e+03   7.91000000e+01   1.02122000e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1093.0
[  1.09300000e+03   7.73000000e+01   9.93090000e-01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.09300000e+03   7.73000000e+01   9.93090000e-01 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
PTID: 1094.0
[  1.09400000e+03   5.71000000e+01   1.20455550e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.09400000e+03   5.71000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1095.0
[  1.09500000e+03   8.06000000e+01   1.16248000e+00 ...,   1.0000000

   1.00000000e+00   0.00000000e+00]
PTID: 1348.0
[  1.34800000e+03   7.96000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.34800000e+03   7.96000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1349.0
[  1.34900000e+03   8.77000000e+01   1.20455550e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.34900000e+03   8.77000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1350.0
[  1.35000000e+03   7.06000000e+01   1.25490000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.35000000e+03   7.06000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1351.0
[  1.35100000e+03   6.08000000e+01   1.28108000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.35100000e+03   6.08000000e+01   1.28108000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
PTID: 1352.0
[  1.35200000e+03  

   0.00000000e+00   1.00000000e+00]
[  1.61500000e+03   5.95000000e+01   1.42194000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
PTID: 1616.0
[  1.61600000e+03   8.53000000e+01   1.15241000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.61600000e+03   8.53000000e+01   1.15241000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
PTID: 1617.0
[  1.61700000e+03   8.16000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.61700000e+03   8.16000000e+01   9.81410000e-01 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1618.0
[  1.61800000e+03   7.64000000e+01   1.26867000e+00 ...,   1.00000000e+00
   0.00000000e+00   1.00000000e+00]
[  1.61800000e+03   7.64000000e+01   1.20455550e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
PTID: 1619.0
[  1.61900000e+03   7.05000000e+01   1.12458000e+00 ...,   0.00000000e+00
   1.00000000e+00   0.00000000e+00]
[  1.61900000e+03   7.05000000e+

In [461]:
#last_two_visits_df[i][j], where i = ptid-1, j=time step where j=0=(t-1), j=1=t
num_ptids = last_two_visits_df.shape[0]
num_feats = Data.shape[1]


t_1_ptid_feats = np.zeros((num_ptids,num_feats-1))
t_ptid_feats = np.zeros((num_ptids,num_feats-1))


for key, val in last_two_visits_time_series_dict.items():
    ind = int(key-1)
    t_1_ptid_feats[ind]=val[0]
    t_ptid_feats[ind] = val[1]

cats = list(Data.columns)
cats.remove('Elapsed Days')

time_shifted_cats = []
for i in range(len(cats)):
    if cats[i] == 'PTID_Key':
        time_shifted_cats.append(cats[i])
    else:
        time_shifted_cats.append(str(cats[i])+'_t-1')
        time_shifted_cats.append(str(cats[i])+'_t')

In [462]:
print(time_shifted_cats[0])

PTID_Key


# MODIFY INPUT VECTOR ACCORDING TO TRAIN, TEST AND 

In [463]:
time_shifted_feat_matrix = np.zeros((num_ptids, 1786*2-1))
for i in range(1786):
    if i==0:
        time_shifted_feat_matrix[:,i] = t_ptid_feats[:,i]
    else:
        #t-1
        time_shifted_feat_matrix[:,2*i-1] = t_1_ptid_feats[:,i]
        #t
        time_shifted_feat_matrix[:,2*i] = t_ptid_feats[:,i]



In [464]:
#modify input for training
Xts_train_data = np.zeros((len(ptid_train), 1786*2-1))
for i in range(len(ptid_train)):
    ptid_index = ptid_train[i]-1
    Xts_train_data[i] = time_shifted_feat_matrix[ptid_index]
Xts_train_df = pd.DataFrame(data=Xts_train_data, columns=time_shifted_cats)

#modify input for testing
Xts_test_data = np.zeros((len(ptid_test), 1786*2-1))
for i in range(len(ptid_test)):
    ptid_index = ptid_test[i]-1
    Xts_test_data[i] = time_shifted_feat_matrix[ptid_index]
Xts_test_df = pd.DataFrame(data=Xts_test_data, columns=time_shifted_cats)

#modify input for validation
Xts_target_data = np.zeros((len(ptid_validation), 1786*2-1))
for i in range(len(ptid_validation)):
    ptid_index = ptid_validation[i]-1
    Xts_target_data[i] = time_shifted_feat_matrix[ptid_index]
Xts_target_df = pd.DataFrame(data=Xts_target_data, columns=time_shifted_cats)

In [12]:
'''#input S must be dictionary w/ 'PTID_Key' as one of keys
#maps patient ids tomost recent (last appearing index in matrix )
# returns dictionary with
# key: PT_ID         val: last index row index
def get_last_patient_visit(S):
    hist_dict={}http://localhost:8888/notebooks/TADPOLE_project.ipynb#
    for pt_id in S['PTID_Key']: #get indexes of last patient visit per patient id
        pt_id_appearances = np.where(S['PTID_Key']==pt_id)
        hist_dict[int(pt_id)] = np.max(pt_id_appearances)
    return hist_dict

#condense dataset into 1 visit per patient (last visit for each patient)
def make_last_visit(dataset, last_visit_dict):
    try:
        last_visit_dataset = np.zeros( (len(last_visit_dict.keys()), dataset.shape[1]) )
    except:
        last_visit_dataset = np.zeros( (len(last_visit_dict.keys()), ))
    ind = 0
    for k in last_visit_dict.keys():
        last_visit_index = last_visit_dict[k]
        last_visit_dataset[ind] = dataset[last_visit_index]
        ind+=1
    return last_visit_dataset

#X and y are np arrays, X/y_last is dict of PTID -> last row index in their respective datasets
#converts X so that it has same # of patients as y and each row corresponds to the same patient
def modify_X_to_y(X, X_last, y_last): 
    y_PT_ID = y_last.keys()
    #print(y_PT_ID)
    modified_X = np.zeros((len(y_PT_ID), X.shape[1]))
    X_PTID_Index = list(X[:,0]) #index here corresponds to vector's index in matrix
    index = 0
    for pt_id in y_PT_ID:
        #print pt_id
        corr_X_index = X_PTID_Index.index(pt_id) #PT_ID -> X_index = PT_ID-1
        modified_X[index] = X[corr_X_index]
        index+=1
    return modified_X

#Ventricles_norm is computed as "Ventricles" divided by "ICV"
#DXCHANGE = {1, 7, 9} encodes healthy control, DXCHANGE = {2, 4, 8} encodes MCI, and 
#DXCHANGE = {3, 5, 6} encodes Alzheimer's diagnosis.
def DXCHANGE_to_diagnosis(DXCHANGE_col):
    expanded_DXCHANGE = np.zeros((len(DXCHANGE_col), 3))
    healthy = set([1,7,9])
    mci = set([2,4,8])
    ad = set([3,5,6])
    for i in range(len(DXCHANGE_col)):
        if int(DXCHANGE_col[i]) in healthy:
            new_row = np.array([1,0,0])
            expanded_DXCHANGE[i] = new_row
        elif int(DXCHANGE_col[i]) in mci:
            new_row = np.array([0,1,0])
            expanded_DXCHANGE[i] = new_row
        
        elif int(DXCHANGE_col[i]) in ad:
            new_row = np.array([0,0,1])
            expanded_DXCHANGE[i] = new_row
    return expanded_DXCHANGE


def diagnosis_to_classes(y_diagnosis_cols):
    condensed_classes = np.zeros((y_diagnosis_cols.shape[0],))
    for i in range(len(condensed_classes)):
        check_equivalence_healthy = y_diagnosis_cols[i] == np.array([1,0,0])
        check_equivalence_mci = y_diagnosis_cols[i] == np.array([0,1,0])
        check_equivalence_ad = y_diagnosis_cols[i] == np.array([0,0,1])
        if check_equivalence_healthy.all() == True:
            condensed_classes[i] = 0
        elif check_equivalence_mci.all() == True:
            condensed_classes[i] = 1
        elif check_equivalence_ad.all() == True:
            condensed_classes[i] = 2        
    return condensed_classes
#converts one hot encoding of diagnosis to one column w/ classes 0-healthy, 1-MCI, 2-AD
def classes_to_diagnosis(y_diagnosis_col):
    condensed_classes = np.zeros((y_diagnosis_col.shape[0], 3))
    for i in range(len(condensed_classes)):
        #check_equivalence_healthy = y_diagnosis_cols[i] == np.array([1,0,0])
        #check_equivalence_mci = y_diagnosis_cols[i] == np.array([0,1,0])
        #check_equivalence_ad = y_diagnosis_cols[i] == np.array([0,0,1])
        if y_diagnosis_col[i] == 0:
            condensed_classes[i] = np.array([1,0,0])
        elif y_diagnosis_col[i] == 1:
            condensed_classes[i] = np.array([0,1,0])
        elif y_diagnosis_col[i] == 2:
            condensed_classes[i] = np.array([0,0,1])     
    return condensed_classes
                                                            
    '''

'''#get dictionary for last visits for corresponding datasets
X_last_visit_dict = get_last_patient_visit(X_cat_to_dataset_dict)
y_train_last_visit_dict = get_last_patient_visit(Y_train_cat_to_dataset_dict)
y_test_last_visit_dict = get_last_patient_visit(Y_test_cat_to_dataset_dict)

#modify datasets so that there is only one visit per ptid
X_last_visit_dataset = make_last_visit(numX_with_date, X_last_visit_dict)
y_train_last_visit_dataset = make_last_visit(y_train, y_train_last_visit_dict)
y_test_last_visit_dataset = make_last_visit(labels_test.values, y_test_last_visit_dict)'''

'''X_train = modify_X_to_y(numX_with_date, X_last_visit_dict, y_train_last_visit_dict)
X_test = modify_X_to_y(numX_with_date, X_last_visit_dict, y_test_last_visit_dict)'''

# Scaling Features

In [465]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
sc = StandardScaler()
X_train_scaled = sc.fit_transform(Xts_train_df.values[:,1:])
X_test_scaled = sc.fit_transform(Xts_test_df.values[:,1:])
X_validation_scaled = sc.fit_transform(Xts_target_df.values[:,1:])

# Implementing Support Vector Classifier/Regression

In [466]:
#create ptid -> years for test and validation set
test_ptid_prediction = {}
target_ptid_prediction = {}
for index, row in labels_test.iterrows():
    if row[0] not in test_ptid_prediction.keys():
        test_ptid_prediction[row[0]] = list()
    test_ptid_prediction[row[0]].append(row[1]) #add year to prediction

for index, row in target.iterrows():
    if row[1] not in target_ptid_prediction.keys():
        target_ptid_prediction[row[1]] = list()
    target_ptid_prediction[row[1]].append(row[0]) #add year to prediction
    

###### MMSE

In [467]:
from sklearn import svm

mmse_models = [svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR()]
y_mmse_t = y_train_ts_df.values[:,31]
y_mmse_tp1 = y_train_ts_df.values[:,32]
y_mmse_tp2 = y_train_ts_df.values[:,33]
y_mmse_tp3 = y_train_ts_df.values[:,34]
y_mmse_tp4 = y_train_ts_df.values[:,35]
y_mmse_tp5 = y_train_ts_df.values[:,36]

mmse_models[0].fit(X_train_scaled, y_mmse_t)
mmse_models[1].fit(X_train_scaled, y_mmse_tp1)
mmse_models[2].fit(X_train_scaled, y_mmse_tp2)
mmse_models[3].fit(X_train_scaled, y_mmse_tp3)
mmse_models[4].fit(X_train_scaled, y_mmse_tp4)
mmse_models[5].fit(X_train_scaled, y_mmse_tp5)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

###### Ventricle Norms

In [535]:
from sklearn import svm

vn_models = [svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR()]
y_vn_t = y_train_ts_df.values[:,25]
y_vn_tp1 = y_train_ts_df.values[:,26]
y_vn_tp2 = y_train_ts_df.values[:,27]
y_vn_tp3 = y_train_ts_df.values[:,28]
y_vn_tp4 = y_train_ts_df.values[:,29]
y_vn_tp5 = y_train_ts_df.values[:,30]

vn_models[0].fit(X_train_scaled, y_vn_t)
vn_models[1].fit(X_train_scaled, y_vn_tp1)
vn_models[2].fit(X_train_scaled, y_vn_tp2)
vn_models[3].fit(X_train_scaled, y_vn_tp3)
vn_models[4].fit(X_train_scaled, y_vn_tp4)
vn_models[5].fit(X_train_scaled, y_vn_tp5)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [534]:
y_train_ts_df.columns[30]

'Ventricles_Norm_t+5'

###### Diagnosis

In [498]:
def diagnosis_to_classes(y_diagnosis_cols):
    condensed_classes = np.zeros((y_diagnosis_cols.shape[0],))
    for i in range(len(condensed_classes)):
        check_equivalence_healthy = y_diagnosis_cols[i] == np.array([1,0,0])
        check_equivalence_mci = y_diagnosis_cols[i] == np.array([0,1,0])
        check_equivalence_ad = y_diagnosis_cols[i] == np.array([0,0,1])
        if check_equivalence_healthy.all() == True:
            condensed_classes[i] = 0
        elif check_equivalence_mci.all() == True:
            condensed_classes[i] = 1
        elif check_equivalence_ad.all() == True:
            condensed_classes[i] = 2        
    return condensed_classes

# #converts one hot encoding of diagnosis to one column w/ classes 0-healthy, 1-MCI, 2-AD
# def classes_to_diagnosis(y_diagnosis_col):
#     condensed_classes = np.zeros((1, 3))
#     for i in range(len(condensed_classes)):
#         #check_equivalence_healthy = y_diagnosis_cols[i] == np.array([1,0,0])
#         #check_equivalence_mci = y_diagnosis_cols[i] == np.array([0,1,0])
#         #check_equivalence_ad = y_diagnosis_cols[i] == np.array([0,0,1])
#         if y_diagnosis_col[i] == 0:
#             condensed_classes[i] = np.array([1,0,0])
#         elif y_diagnosis_col[i] == 1:
#             condensed_classes[i] = np.array([0,1,0])
#         elif y_diagnosis_col[i] == 2:
#             condensed_classes[i] = np.array([0,0,1])     
#     return condensed_classes

def classes_to_diagnosis(y_diagnosis_col):
    condensed_classes = np.zeros((1, 3))
    #check_equivalence_healthy = y_diagnosis_cols[i] == np.array([1,0,0])
    #check_equivalence_mci = y_diagnosis_cols[i] == np.array([0,1,0])
    #check_equivalence_ad = y_diagnosis_cols[i] == np.array([0,0,1])
    if y_diagnosis_col == 0:
        condensed_classes = np.array([1,0,0])
    elif y_diagnosis_col == 1:
        condensed_classes = np.array([0,1,0])
    elif y_diagnosis_col == 2:
        condensed_classes = np.array([0,0,1])     
    return condensed_classes

In [470]:
diagnosis_models = [svm.SVC(), svm.SVC(), svm.SVC(), svm.SVC(), svm.SVC(), svm.SVC()]
y_diag_t = np.column_stack( (y_train_ts_df.values[:,1], y_train_ts_df.values[:,7], y_train_ts_df.values[:,13]) )
y_diag_tp1 = np.column_stack( (y_train_ts_df.values[:,2], y_train_ts_df.values[:,8], y_train_ts_df.values[:,14]) )
y_diag_tp2 = np.column_stack( (y_train_ts_df.values[:,3], y_train_ts_df.values[:,9], y_train_ts_df.values[:,15]) )
y_diag_tp3 = np.column_stack( (y_train_ts_df.values[:,4], y_train_ts_df.values[:,10], y_train_ts_df.values[:,16]) )
y_diag_tp4 = np.column_stack( (y_train_ts_df.values[:,5], y_train_ts_df.values[:,11], y_train_ts_df.values[:,17]) )
y_diag_tp5 = np.column_stack( (y_train_ts_df.values[:,6], y_train_ts_df.values[:,12], y_train_ts_df.values[:,18]) )

y_diag_t_classes = diagnosis_to_classes(y_diag_t)
y_diag_tp1_classes = diagnosis_to_classes(y_diag_tp1)
y_diag_tp2_classes = diagnosis_to_classes(y_diag_tp2)
y_diag_tp3_classes = diagnosis_to_classes(y_diag_tp3)
y_diag_tp4_classes = diagnosis_to_classes(y_diag_tp4)
y_diag_tp5_classes = diagnosis_to_classes(y_diag_tp5)

diagnosis_models[0].fit(X_train_scaled, y_diag_t_classes)
diagnosis_models[1].fit(X_train_scaled, y_diag_tp1_classes)
diagnosis_models[2].fit(X_train_scaled, y_diag_tp2_classes)
diagnosis_models[3].fit(X_train_scaled, y_diag_tp3_classes)
diagnosis_models[4].fit(X_train_scaled, y_diag_tp4_classes)
diagnosis_models[5].fit(X_train_scaled, y_diag_tp5_classes)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

###### ADAS13

In [471]:

adas13_models = [svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR(),svm.SVR()]
y_adas13_t = y_train_ts_df.values[:,19]
y_adas13_tp1 = y_train_ts_df.values[:,20]
y_adas13_tp2 = y_train_ts_df.values[:,21]
y_adas13_tp3 = y_train_ts_df.values[:,22]
y_adas13_tp4 = y_train_ts_df.values[:,23]
y_adas13_tp5 = y_train_ts_df.values[:,24]

adas13_models[0].fit(X_train_scaled, y_adas13_t)
adas13_models[1].fit(X_train_scaled, y_adas13_tp1)
adas13_models[2].fit(X_train_scaled, y_adas13_tp2)
adas13_models[3].fit(X_train_scaled, y_adas13_tp3)
adas13_models[4].fit(X_train_scaled, y_adas13_tp4)
adas13_models[5].fit(X_train_scaled, y_adas13_tp5)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# Making Prediction CSV

In [536]:
#make prediction matrices
predicted_diagnosis_test = []
predicted_adas13_test = []
predicted_mmse_test = []
predicted_vn_test = []

predicted_diagnosis_valid = []
predicted_adas13_valid = []
predicted_mmse_valid = []
predicted_vn_valid = []
for i in range(6):
    predicted_diagnosis_test.append(diagnosis_models[i].predict(X_test_scaled))
    predicted_adas13_test.append(adas13_models[i].predict(X_test_scaled))
    predicted_mmse_test.append(mmse_models[i].predict(X_test_scaled))
    predicted_vn_test.append(vn_models[i].predict(X_test_scaled))
    
    predicted_diagnosis_valid.append(diagnosis_models[i].predict(X_validation_scaled))
    predicted_adas13_valid.append(adas13_models[i].predict(X_validation_scaled))
    predicted_mmse_valid.append(mmse_models[i].predict(X_validation_scaled))
    predicted_vn_valid.append(vn_models[i].predict(X_validation_scaled))

In [473]:
#PTID indexes in the prediction matrices 
X_test_indices = list(Xts_test_df['PTID_Key'])
X_valid_indices = list(Xts_target_df['PTID_Key']) 

In [482]:
#get original dates of patient visits to create csv
original_test_dates_df = pd.read_csv('TADPOLE_TargetData_test.csv')
original_validation_dates_df = pd.read_csv('TADPOLE_PredictTargetData_valid.csv')

#original_test_dates_df['Date'] = pd.to_datetime(original_test_dates_df['Date'],format="%Y-%m-%d",errors='coerce')
#original_validation_dates_df['Date'] = pd.to_datetime(original_validation_dates_df['Date'],format="%m/%d/%y",errors='coerce')

#keys to iterate over in the dictionaries in order
original_test_order = []
original_validation_order = []
for index, row in original_test_dates_df.iterrows():
    if row[1] not in original_test_order:
        original_test_order.append(row[1])
        
for index, row in original_validation_dates_df.iterrows():
    if row[1] not in original_validation_order:
        original_validation_order.append(row[1])

In [431]:
print(original_test_dates_df.columns)
print(original_validation_dates_df.columns)

Index(['Date', 'PTID_Key', 'CN_Diag', 'MCI_Diag', 'AD_Diag', 'ADAS13',
       'Ventricles_Norm', 'MMSE'],
      dtype='object')
Index(['Date', 'PTID_Key', 'CN_Diag', 'MCI_Diag', 'AD_Diag', 'ADAS13',
       'Ventricles_Norm', 'MMSE'],
      dtype='object')


In [484]:
type(original_test_dates_df['Date'][0])

str

In [491]:
predicted_diagnosis_test[int(timestep)][corr_ptid_index].shape

()

In [537]:
#indices to set results matrix
ind_test = 0 
ind_valid = 0

#np matrices to hold results
final_prediction_test = np.zeros( (original_test_dates_df.shape[0],7) )
final_prediction_validation = np.zeros( (original_validation_dates_df.shape[0], 7) ) 

#iterate through keys in order they appear in original csv
for key in original_test_order:
    #get list of all timesteps to predict
    timesteps_to_predict = test_ptid_prediction[key]
    #the the corresponding index for the prediction matrix
    corr_ptid_index = X_test_indices.index(key)
    #print('PTID', key)
    #print('corresponding index', corr_ptid_index)
    for timestep in timesteps_to_predict:
        #print('timestep', timestep)
    
        prediction_entry = np.zeros((7,))
        #prediction_entry[0] = str(original_test_dates_df['Date'][ind_test])
        prediction_entry[0] = key
        prediction_entry[1:4] = classes_to_diagnosis(predicted_diagnosis_test[int(timestep)][corr_ptid_index])
        prediction_entry[4] = predicted_adas13_test[int(timestep)][corr_ptid_index]
        prediction_entry[5] = predicted_vn_test[int(timestep)][corr_ptid_index]
        prediction_entry[6] = predicted_mmse_test[int(timestep)][corr_ptid_index]
        
        final_prediction_test[ind_test] = prediction_entry
        ind_test += 1
        

#iterate through keys in order they appear in original csv
for key in original_validation_order:
    #get list of all timesteps to predict
    timesteps_to_predict = target_ptid_prediction[key]
    #the the corresponding index for the prediction matrix
    corr_ptid_index = X_valid_indices.index(key)
    for timestep in timesteps_to_predict:
    
        prediction_entry = np.zeros((7,))
        #prediction_entry[0] = str(original_validation_dates_df['Date'][ind_valid])
        prediction_entry[0] = key
        prediction_entry[1:4] = classes_to_diagnosis(predicted_diagnosis_valid[int(timestep)][corr_ptid_index])
        prediction_entry[4] = predicted_adas13_valid[int(timestep)][corr_ptid_index]
        prediction_entry[5] = predicted_vn_valid[int(timestep)][corr_ptid_index]
        prediction_entry[6] = predicted_mmse_valid[int(timestep)][corr_ptid_index]
        
        final_prediction_validation[ind_valid] = prediction_entry
        ind_valid += 1
        

In [538]:
print(final_prediction_test[21])


[  1.62800000e+03   1.00000000e+00   0.00000000e+00   0.00000000e+00
   1.45729092e+01   4.12216000e-02   2.71642846e+01]


In [523]:
original_test_dates_df

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2014-01-02,1603,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-15,1603,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-10-29,1603,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-08-11,1603,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-18,1603,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-02-07,955,0.0,1.0,0.0,14.0,0.028088,29.0
6,2013-07-31,955,NaN,NaN,NaN,NaN,NaN,NaN
7,2014-01-29,955,0.0,1.0,0.0,12.0,0.028790,29.0
8,2015-01-27,955,0.0,1.0,0.0,14.0,0.031549,29.0
9,2016-01-26,955,0.0,1.0,0.0,15.0,NaN,29.0


In [381]:
original_test_dates_df

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2013-02-13,5,1.0,0.0,0.0,16.0,0.019279,28.0
1,2013-08-14,5,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-02-20,5,0.0,1.0,0.0,23.0,NaN,27.0
3,2015-03-05,5,0.0,0.0,1.0,29.0,NaN,27.0
4,2016-04-28,5,0.0,0.0,1.0,26.0,NaN,19.0
5,2014-04-07,20,0.0,1.0,0.0,25.0,0.046655,25.0
6,2013-09-30,20,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-04-04,20,0.0,1.0,0.0,25.0,0.042373,26.0
8,2013-09-12,30,1.0,0.0,0.0,8.0,0.033172,30.0
9,2014-03-26,30,NaN,NaN,NaN,NaN,NaN,NaN


# Classification Error

In [ ]:
def get_accuracy(generated_csv, given_csv):
    my_prediction = pd.read_csv(generated_csv)
    ground_truth = pd.read_csv(given_csv)
    valid_index = list()
    
    #get list of valid indices to compare
    for index, row in ground_truth.iterrows():
        diagnoses = row[2:5]
        valid_entry = np.isnan(diagnoses)
        if(list(valid_entry).count(False)==len(diagnoses)):
            valid_index.append(index)
        
    #iterate through valid indicies and compare classification error
    num_instances = len(valid_index)
    num_correct = 0
    
    for index in valid_index:
        yhat = my_prediction.values[index]
        y = ground_truth.values[index]
        if(yhat[2:5] == y[2:5]):
            num_correct+=1
    return float(num_correct/num_instances)

#def get_training_loss()